In [1]:
from stravaio import strava_oauth2,StravaIO
import pandas as pd
import json
import matplotlib as plt
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"] = (30, 10)
sns.set(rc={'figure.figsize':(30,10)})


In [3]:
oauth_obj = strava_oauth2(client_id=90994, client_secret="ed89e268fddb96e2a7aee3a2e13cbcc757c06283")
print(oauth_obj["access_token"])

2022-12-03 13:42:48.929 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2022-12-03 13:42:53.986 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: c210d413e88cfde985eee53306dfd76b4e002f77
2022-12-03 13:42:54.149 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: f323415afa8977b09464adacc6d72617a7c58062


f323415afa8977b09464adacc6d72617a7c58062


In [ ]:
client = StravaIO(access_token=oauth_obj["access_token"])

In [ ]:
list_activities = client.get_logged_in_athlete_activities(after='10-01-2022')

In [ ]:
def get_activity_stream_dataframe(athlete_id =94046257 ,id=8146797672):
    streams = client.get_activity_streams(athlete_id =athlete_id ,id=id)
    df = pd.DataFrame(streams.to_dict())
    #act_stream["efficiency"] = act_stream["velocity_smooth"] / act_stream["heartrate"]
    return df

In [ ]:
ath = client.get_logged_in_athlete()
ath

In [ ]:
df = pd.DataFrame()
for act in list_activities:
    act_df = pd.json_normalize(act.to_dict())
    if act.type != 'VirtualRun':
        act_stream_df = get_activity_stream_dataframe(athlete_id =act.athlete.id ,id=act.id)
        act_df.loc[0,"heartrate"] = act_stream_df.heartrate.mean()
    df = pd.concat([df, act_df])    
df

In [ ]:
df = df.set_index("id")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["moving_time_minute"] = df["moving_time"] /  60 
df["distance_km"] = df["distance"] /  1000


In [ ]:
df.plot(y="distance_km",x="moving_time_minute", kind='scatter',c="heartrate",colormap='coolwarm',s=50,figsize=(30, 10))


In [ ]:
df_2022 = df.loc[df.start_date_local  > "2022-01-06",['start_date_local',"distance","moving_time"]]

In [ ]:
df_2022_week_summary = df_2022.groupby([df_2022['start_date_local'].dt.isocalendar().week]).agg({'distance': ['sum', 'max']})

In [ ]:
df_2022_week_summary_km = df_2022_week_summary / 1000

In [ ]:
df_2022_week_summary_km.plot.bar(figsize=(30, 10))

In [ ]:
sns.scatterplot(data=df,x="average_speed", y="heartrate",size="distance",hue="start_date",sizes=(20, 2000),palette='coolwarm')

In [ ]:
act_df = get_activity_stream_dataframe()
act_df.describe()

In [ ]:
act_df.loc[:,["time","velocity_smooth","heartrate","moving","altitude"]].plot( figsize=(30, 10),x="time")

In [ ]:
act_df.loc[:,["time","velocity_smooth","heartrate","moving","grade_smooth","altitude"]].plot( figsize=(30, 10),x="time")

In [ ]:
sns.lineplot(data=act_df.loc[:,["velocity_smooth","heartrate","moving","altitude"]], palette="tab10", linewidth=2.5)